# Exploring the Galaxy of Authorization: Setting Up RBAC, Managing User Roles, and Creating Authorization Middleware
## Introduction to Authorization

Greetings! Today, we'll unlock the world of Authorization, with a focus on user roles, Role-Based Access Control (RBAC), and authorization middleware.

To illustrate Authorization simply—it's like a space mission, where only authorized astronauts can enter restricted areas. Now that we understand Authorization, let's implement it in our applications!
## Understanding Authorization and Role-Based Access Control (RBAC)

Authorization comes into play post-authentication, deciding what actions authenticated users can perform. To break it down, Authentication is a spaceship's identity check, while Authorization is the spacesuit that guides users to access the various compartments of the spaceship.

Imagine a space facility with scientists and engineers, each having defined roles and access levels. Similarly, the Role-Based Access Control (RBAC) system assigns permissions based on roles.

Take an e-commerce system as an example:

    Buyer views products.
    Seller manages their products.
    Admin controls the complete system.

RBAC is essential for managing large system!

## Managing User Roles

In an authorization system, user roles dictate their system access. With MongoDB and Mongoose, we can manage user roles.

    Creating User Roles: Consider 'User' and 'Admin' roles. To oversee our system, we design a user schema with role as a property.


In [ ]:
import mongoose from 'mongoose';

const UserSchema = mongoose.Schema({
    username: String,
    password: String,
    role: {
        type: String,
        default: 'User',
    },
});


    Assigning Permissions to Roles: Each role is given permissions. For instance, the 'User' role only allows the 'Read' operation, while the 'Admin' role can 'Read', 'Write', and 'Delete'.
    Assigning Users to Roles: Users are assigned roles upon signing up.

## Establishing Authorization Middleware

Middleware in Express.js are vital functions that have access to the request and response objects, as well as the next middleware function in the application’s request-response cycle.

In the context of authorization, middleware come in particularly handy to check and verify the roles of users, in order to manage access rights to certain routes. Let's build an authorization middleware that will only allow 'Admin' role users to access a specific route.


In [ ]:
const users = [
  { username: 'Alice', role: 'Admin' },
  { username: 'Bob', role: 'User' }
]; // An imaginary list of users for demonstration purpose

function checkifLoggedin(req) {
  return users[0];  // For the simplicity of this demo, we return the first user from our list, assuming they are logged in
} // Fake function just for the demonstration

function checkUserRole(req, res, next) {
  const user = checkifLoggedin(req);
  if (user.role !== 'Admin') {
    return res.status(403).send('You are not authorized to access this resource');
  }
  req.user = user; 
  return next(); 
}; // Middleware function to validate user roles

app.use('/admin', checkUserRole); // only apply this middleware to '/admin' route
app.get('/admin', function (req, res) {
  res.send(`Welcome ${req.user.username}! You are inside the Admin route`);
});

The checkUserRole middleware function checks if the logged-in user's role is 'Admin'. If so, it then transfers control to the next middleware function in the queue, which in this case is the route handler for '/admin'. If the user role isn't 'Admin', the server responds with a '403 Forbidden' status message.

Now, if you navigate to the /admin endpoint, only 'Admin' users will be able to access it. Anyone else will get a '403 Forbidden' message. As you continue building more complex applications, this mechanism of using middleware for role-based access control will become an essential aspect of maintaining application security. Welcome to an essential part of managing authorization in your web applications!

## Utilizing MongoDB in Authorization Middleware

We have already set up an authorization middleware earlier. However, in real life applications, user data including roles are typically stored in a database. For this, we can use MongoDB and Mongoose.

    Creating User Roles in MongoDB: First, let's create a user model with Mongoose which includes user roles.


In [ ]:
import mongoose from 'mongoose';

const UserSchema = new mongoose.Schema({
  username: String,
  password: String,
  role: {
    type: String,
    default: 'User'
  },
});

const User = mongoose.model('User', UserSchema);

Setting up Authorization Middleware with MongoDB models: Then, let's set up a middleware to authorize different user roles. The middleware will retrieve user information from the database and verify before granting the access:

In [ ]:
import express from 'express';
const app = express();

// Middleware function to authorize user roles
function verifyUserRole(req, res, next) {
  User.findById(req.user._id)
    .then(user => {
      // Check if user role is 'Admin'
      if (user.role === 'Admin') {
        next();
      } else {
        res.status(403).send('You are not authorized to access this resource');
      }
    })
    .catch(err => {
      console.error(err);
      res.status(500).send('Internal Server Error');
    });
}

app.use(verifyUserRole);

In the provided example, verifyUserRole is a middleware function that checks the role of the authenticated user stored in the MongoDB database. If the user's role is 'Admin', the middleware calls next(), which passes control to the next middleware function. Otherwise, it sends the 403 status code (representing 'Forbidden'). Additionally, if there is an error during this process, such as a failure to connect with the database, it handles the error gracefully by returning a 500 Internal Server Error response.

As a result, our Express.js application, armed with MongoDB and Mongoose, becomes more adept at authorizing different user roles based on database data. This empowers us to manage user permissions more diversely and effectively.

## Lesson Summary and Next Steps

Our journey into the Authorization galaxy was thrilling. We explored the concepts of authorization, role-based access control systems, managing user roles, and creating authorization middleware.

Looking ahead, we're preparing to delve into error handling. Filled with hands-on practice, our journey continues to impart skills and take us deeper into the Universe of Authorization! Until next time, farewell!


## Exercises

Galactic Pioneer, let's elevate our security! Modify the code so that a Scientist can read data from the space station. Remember, authorization verifies roles before access is granted.

In [ ]:
// index.js (Express server) - before francis complete
const express = require('express');
const cors = require('cors');
const app = express();

// Enable CORS for all routes
app.use(cors());

// Simulated user data for demonstration purposes
const users = [
  { id: 1, username: 'astronaut_alice', role: 'Engineer' },
  { id: 2, username: 'astronaut_bob', role: 'Scientist' }
];

// Middleware to check for space station data access based on user role
const checkRoleAccess = (allowedRoles) => (req, res, next) => {
  const { userId } = req.params;
  const user = users.find(u => u.id.toString() === userId);

  if (user && allowedRoles.includes(user.role)) {
    next();
  } else {
    res.status(403).send('Access denied: You do not have the required clearance.');
  }
};

app.get('/space-station-data/:userId', checkRoleAccess(['Engineer']), (req, res) => {
  res.send('Space Station data accessed successfully.');
});

app.listen(5000, () => console.log('Server running on port 5000'));

In [ ]:
// index.js (Express server) - after francis complete
const express = require('express');
const cors = require('cors');
const app = express();

// Enable CORS for all routes
app.use(cors());

// Simulated user data for demonstration purposes
const users = [
  { id: 1, username: 'astronaut_alice', role: 'Engineer' },
  { id: 2, username: 'astronaut_bob', role: 'Scientist' }
];

// Middleware to check for space station data access based on user role
const checkRoleAccess = (allowedRoles) => (req, res, next) => {
  const { userId } = req.params;
  const user = users.find(u => u.id.toString() === userId);

  if (user && allowedRoles.includes(user.role)) {
    next();
  } else {
    res.status(403).send('Access denied: You do not have the required clearance.');
  }
};

app.get('/space-station-data/:userId', checkRoleAccess(['Engineer', 'Scientist']), (req, res) => { // added Scientist role to checkRoleAccess
  res.send('Space Station data accessed successfully.');
});

app.listen(5000, () => console.log('Server running on port 5000'));

In [ ]:
// App.js (React client)
import axios from 'axios';
import React, { useState } from 'react';

const App = () => {
  const [errorMessage, setErrorMessage] = useState('');
  const [successMessage, setSuccessMessage] = useState('');
  // IDs of the users trying to access the Space Station
  const userIDs = [1, 2]; 
  
  React.useEffect(() => {
    userIDs.forEach(userID => {
      axios.get(`/space-station-data/${userID}`)
        .then(response => setSuccessMessage(`User ${userID} response – HTML document returned: ${response.data}`))
        .catch(error => {
          if (error.response) {
            if (error.response.status === 403) {
              setErrorMessage(`User ${userID}: ${error.response.data}`);
            } else if (error.response.status === 404) {
              setErrorMessage(`User ${userID}: ${error.response.data}`);
            }
          } else {
            setErrorMessage(`User ${userID}: Server not responding.`);
          }
        });
    });
  }, []);

  return (
    <div>
      <h1>Space Station Operations</h1>
      <pre className="success">{successMessage}</pre>
      <pre className="error">{errorMessage}</pre>
    </div>
  );
};

export default App;

Galactic Pioneer, it's time to change the role of Lieutenant Kerrigan. In the server's user list, update her role from User to Admin. This will grant her the appropriate access level to the spaceship's control panel.

In [ ]:
// index.js (Express server) - francis complete
const express = require('express');
const cors = require('cors');

// Create a new express app
const app = express();

// Middleware for parsing JSON and urlencoded form data
app.use(express.json());
app.use(express.urlencoded({ extended: false }));
app.use(cors());
const port = 5000;

const users = [
  { id: 1, name: 'Captain Raynor', role: 'Admin' },
  { id: 2, name: 'Lieutenant Kerrigan', role: 'Admin' }  // Role needs to be changed - FQ changed this from "User" to "Admin"
];

app.get('/api/check-access', (req, res) => {
  const user = users.find(u => u.name === 'Lieutenant Kerrigan');
  if (user.role === 'Admin') {
    res.json({ message: 'Access granted to control panel.' });
  } else {
    res.status(403).json({ message: 'Access denied.' });
  }
});

app.listen(port, () => console.log(`Server running on port ${port}`));

In [ ]:
// App.js (React client) - francis complete
import { useEffect, useState } from 'react';
import axios from 'axios';

const SpaceStationAccess = () => {
  const [message, setMessage] = useState('');

  useEffect(() => {
    axios.get('/api/check-access')
      .then(response => setMessage(response.data.message))
      .catch(error => setMessage('Access denied: ' + (error.response.message ? error.response.data : error.message)));
  }, []);

  return (
    <div>
      <h1>Welcome to the Space Station. Checking Access...</h1>
      {message && <p>{message}</p>}
    </div>
  );
};

export default SpaceStationAccess;

Stellar Navigator, it's time to step up! The space station's restricted area needs safeguarding. Implement middleware to check whether a user is a scientist before granting access. Shine on!

In [ ]:
// index.js (Express server) - final version
const express = require('express');
const cors = require('cors');

// Create a new express app
const app = express();

// Middleware for parsing JSON and urlencoded form data
app.use(express.json());
app.use(express.urlencoded({ extended: false }));
app.use(cors());
const port = 5000;

// Hardcoded list of users with their roles for demonstration purposes
const users = [
  { username: 'astroJane', role: 'Scientist' },
  { username: 'cosmoPilot', role: 'Astronaut' },
  { username: 'spaceBoss', role: 'Admin' },
];

function isScientist(req, res, next) {
  // TODO: Validate if the username from query parameters is a Scientist, if so, proceed with next(), otherwise set the status code to 403 with message indicating that access is denied
  const user = users.find(u => u.username === req.query.username);
  if (user && user.role !== 'Scientist'){
    return res.status(403).send('You are not authorized to access this resource');
  }
  req.user = user;
  return next();
}

// Protect the space experiments data route with the isScientist middleware
app.get('/api/space-experiments', isScientist, (req, res) => {
  res.send('Welcome to the space experiments data!');
});

app.listen(port, () => console.log(`Server running on port ${port}`));

In [ ]:
// App.js (React client) - final version
import axios from 'axios';
import { useState } from 'react';

function App() {
  const [message, setMessage] = useState('');

  const handleAccessSpaceData = (username) => {
    axios.get('/api/space-experiments', { params: { username: username } })
      .then(response => setMessage(response.data))
      .catch(error => {
        const errorMessage = (error.response && error.response.data) ? error.response.data : 'An error occurred!';
        setMessage(errorMessage);
      });
  };

  return (
    <div>
      <button onClick={() => handleAccessSpaceData('astroJane')}>
        Access Space Experiments Data as astroJane
      </button>
      <button onClick={() => handleAccessSpaceData('cosmoPilot')}>
        Access Space Experiments Data as cosmoPilot
      </button>
      <div>{message}</div>
    </div>
  );
}

export default App;

Great work navigating through the stars of authorization, Space Voyager! Your final mission in this lesson is to set up a galaxy gatekeeper on your Express.js server. Imagine a Space Station where Scientists and Engineers can only enter their designated zones. Craft two Express routes with appropriate middleware to check user roles and allow entry only if they match!

In [ ]:
// index.js (Express server) - final version
const express = require('express');
const app = express();
const port = 5000;

const user = { username: 'astro-novice', role: 'Scientist' }; // For simplicity let's assume this is the logged in user

// TODO: Write a middleware function that checks if a user has the 'Scientist' role.
// If they do, let them proceed to the Science Lab.
function isScientist(req, res, next) {
  // TODO: Validate if the username from query parameters is a Scientist, if so, proceed with next(), otherwise set the status code to 403 with message indicating that access is denied
  if (user.role !== 'Scientist'){
    return res.status(403).send('You are not authorized to access this resource');
  }
  req.user = user;
  return next();
}

// TODO: Write another middleware function that checks if a user has the 'Engineer' role.
// If they do, let them proceed to the Engineering Deck.
function isEngineer(req, res, next) {
  // TODO: Validate if the username from query parameters is a Scientist, if so, proceed with next(), otherwise set the status code to 403 with message indicating that access is denied
  if (user.role !== 'Engineer'){
    return res.status(403).send('You are not authorized to access this resource');
  }
  req.user = user;
  return next();
}

// These users are trying to go to exclusive areas in the Space Station.
// TODO: Create two routes: /api/scientist for the Science Lab and /api/engineer for the Engineering Deck.
// Verify the user's role using your middleware functions before granting access.
route_science = '/api/scientist'
route_engineer = '/api/engineer'

app.get(route_science, isScientist, (req, res) => {
  res.send('Welcome to the space experiments data!');
});

app.get(route_engineer, isEngineer, (req, res) => {
  res.send('Welcome to the space experiments data!');
});


// Remember to start your server listening by calling app.listen on your port variable.
app.listen(port, () => console.log(`Server running on port ${port}`));

In [ ]:
//App.js (React client) - final version
import axios from 'axios';
import { useState } from 'react';

export default function App() {
  const [message, setMessage] = useState('');

  const checkAccess = (role) => {
    const roleCapitalized = role.charAt(0).toUpperCase() + role.slice(1);
    axios.get(`/api/${roleCapitalized}`).then(response => {
      setMessage(response.data.message);
    }).catch(error => {
      setMessage('Access denied: ' + error.response.data.error);
    });
  };

  return (
    <div>
      <h1>Hello astro-novice!</h1>
      <button onClick={() => checkAccess('scientist')}>Scientist Area</button>
      <button onClick={() => checkAccess('engineer')}>Engineer Zone</button>
      {message && <p>{message}</p>}
    </div>
  );
}